In [30]:
# upload input video to extract frames
from google.colab import files
input_video = files.upload()

Saving campfire_1920_1080_30fps.mp4 to campfire_1920_1080_30fps.mp4


In [31]:
# check if is accesible
video_filename = list(input_video.keys())[0]
try:
    print(video_filename)
except Exception as e:
    print(f"Error: {e}")

campfire_1920_1080_30fps.mp4


In [32]:
# Extract the neccessary frames
import cv2
import os
from PIL import Image

# function to extract frames from the video
def extract_frames(video_path, num_frames=15, start_time=0, end_time=None, save_folder='extracted_frames'):
    # craete a folder to store extraced frames
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    cap = cv2.VideoCapture(video_path)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps

    if end_time is None:
        end_time = duration

    print(f"Extracting frames from {start_time:.2f} to {end_time:.2f} seconds.")

    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)

    total_extractable_frames = end_frame - start_frame
    frame_interval = total_extractable_frames // num_frames

    frames = []

    for i in range(num_frames):
        # Set the position of the next frame to extract
        frame_position = start_frame + i * frame_interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_position)

        # Read the frame
        ret, frame = cap.read()

        if ret:
            # Convert to a Pillow Image
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)

            frame_filename = f"{save_folder}/frame_{i+1:02d}.png"
            img.save(frame_filename)
            frames.append(frame_filename)

    cap.release()
    return frames

num_frames = 15
start_time = 0
end_time = None
extracted_frames = extract_frames(video_filename, num_frames=num_frames, start_time=start_time, end_time=end_time)
print(f"Extracted {len(extracted_frames)} frames and saved them to the 'extracted_frames' folder.")

Extracting frames from 0.00 to 20.47 seconds.
Extracted 15 frames and saved them to the 'extracted_frames' folder.


In [42]:
# Create gif from those frames
from PIL import Image, ImageOps
import glob
import os

images = []

# Get all the images in the 'extracted_frames' folder
input_folder = './extracted_frames/'
for filename in sorted(glob.glob(os.path.join(input_folder, 'frame_*.png'))):
    im = Image.open(filename)
    old_size = im.size

    border_color = (255, 255, 255)  # white border
    border_thickness = 20

    aspect_ratio = old_size[0] / old_size[1]

    # Resize image to fit the old size while maintaining the aspect ratio
    if old_size[0] > old_size[1]:
        new_width = old_size[0] - 2 * border_thickness
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = old_size[1] - 2 * border_thickness
        new_width = int(new_height * aspect_ratio)

    # Resize the image while maintaining aspect ratio
    resized_im = im.resize((new_width, new_height), Image.Resampling.LANCZOS)

    # Add border around resized image
    new_im = ImageOps.expand(resized_im, border=border_thickness, fill=border_color)

    images.append(new_im)

output_gif_path = './ass5_output.gif'
images[0].save(output_gif_path,
               save_all=True, append_images=images[1:], optimize=False, duration=500, loop=0)

print(f"GIF saved as {output_gif_path}")


GIF saved as ./ass5_output.gif
